In [1]:
import random
import numpy as np
import tensorflow as tf

seed_value= 1111
random.seed(seed_value)
np.random.seed(seed_value)
tf.set_random_seed(seed_value)

import warnings
import sys
import os
import pandas as pd
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
# from sklearn.metrics import mean_absolute_percentage_error
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape, RepeatVector
from keras.layers.merge import concatenate, multiply,add
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from keras.models import load_model
import datetime
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import random as python_random
from statsmodels.tsa.stattools import pacf
from matplotlib import pyplot

/home/chatta/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chatta/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chatta/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chatta/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [2]:
# utility function to make input window for training and test sets

def make_input(data,window_size,horizon=1):
    length=data.shape[0]
    y = np.zeros([length-window_size+1-horizon,horizon])
    output=np.zeros([length-window_size+1-horizon,window_size])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],window_size), y

def make_k_input(data,horizon):
    length = data.shape[0]
    output= np.zeros([length+1-horizon,horizon])
    for i in range(length-horizon+1):
        output[i:i+1,:]=data[i:i+horizon]
    return output.reshape(output.shape[0],horizon)

def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon

    data = np.append(data,np.zeros([horizon-extra]))

    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size), y

def nonov_make_k_input(data,horizon):
    length = data.shape[0]
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],(horizon-extra))
    data = np.append(data,data_app)    

    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon):(i*horizon)+horizon]
    return output.reshape(output.shape[0],horizon)


def metrics(pred,gt):
    l = pred.shape[1]
#     print(l)
    err_mse = np.zeros((l))
    err_mae = np.zeros((l))

    for i in range(l):
        err_mse[i] = mse(pred[:,i],gt[:,i])
        err_mae[i] = mae(pred[:,i],gt[:,i])
        
    return np.sqrt(np.mean(err_mse)),np.mean(err_mae)

def concat(data):
    temp=np.zeros((data.shape[0]*207,data.shape[1]))
    for i in range(207):
        temp[i*data.shape[0]:(i+1)*data.shape[0],:]=data[:,:,i]
    return temp

def window_normalize(data_x,data_y):
    
    min_in = data_x.min(1).reshape(data_x.shape[0],1)
    max_in = data_x.max(1).reshape(data_x.shape[0],1)
    denom = (max_in-min_in)
    a = np.where(denom == 0)[0]
    denom[a] = max_in[a] 
    a = np.where(denom == 0)[0]
    if a.size >0:
        denom[a]=1
    out = (data_x-min_in)/denom
    out=out.reshape(out.shape[0],out.shape[1])
    out_y=(data_y-min_in)/denom
    return out,out_y,denom,min_in

def std_window_normalize(data_x,data_y):
    
    mean_in = np.mean(data_x,axis=1).reshape(data_x.shape[0],1)
    std_in = np.std(data_x,axis=1).reshape(data_x.shape[0],1)
    
    a = np.where(std_in == 0)[0]
    std_in[a] = 1
    
    out = (data_x-mean_in)/std_in
    out=out.reshape(out.shape[0],out.shape[1])
    out_y=(data_y-mean_in)/std_in
    return out,out_y,std_in,mean_in

def p_window_normalize(data_x):
    
    min_in = data_x.min(1).reshape(data_x.shape[0],1)
    max_in = data_x.max(1).reshape(data_x.shape[0],1)
    denom = (max_in-min_in)
    a = np.where(denom == 0)[0]
    denom[a] = max_in[a] 
    a = np.where(denom == 0)[0]
    if a.size >0:
        denom[a]=1
    out = (data_x-min_in)/denom
    out=out.reshape(out.shape[0],out.shape[1])
    
    return out

def p_std_window_normalize(data_x):
    
    mean_in = np.mean(data_x,axis=1).reshape(data_x.shape[0],1)
    std_in = np.std(data_x,axis=1).reshape(data_x.shape[0],1)
    
    a = np.where(std_in == 0)[0]
    std_in[a] = 1
    
    out = (data_x-mean_in)/std_in
 
    out=out.reshape(out.shape[0],out.shape[1])
    
    return out

In [116]:
dataset="nasdaq"
horizon=6

d = "/home/chatta/logic_rules/knowledge_preds/"
data_path="/home/chatta/logic_rules/data/"+dataset
knowledge_pred_path =d+dataset+'/horizon_'+str(horizon)+'/t_preds.csv'
# knowledge_preds = knowledge_preds[:12660,:]
# data1=data
# data = pd.read_csv('/home/chatta/logic_rules/data/traffic_30_with_std.csv',usecols=['avg_flow','std'])
# std_data=np.asarray(data['std'])
# data= np.asarray(data['avg_flow'])
# data1=data[47:]
# horizon = 1
# window_size=3
# knowledge_preds=np.asarray(pd.read_csv("/home/chatta/logic_rules/results/pacf_preds/traffic_30_std/h1/preds.csv",header=None))
# n_train=int(0.8*data.size)+1
# n_test=int(.1*data.size)
# n_val=2*n_test

if dataset=="traffic":
    window_size = 12   
    data = np.asarray(pd.read_csv(data_path+".csv",header=None))
    knowledge_preds = np.asarray(pd.read_csv(knowledge_pred_path,header=None))
    n_val=2880         # index from where validation set starts
    n_test=1440        # index from where test set starts
    data_length = data.shape[1]
    t_size=data.shape[0]
    output = np.zeros((n_test,data_length))
    final_in_train = np.zeros([1,window_size+horizon,1])
    final_in_val =final_in_train
    final_in_test=final_in_train
    final_lbl_train = np.zeros([1,horizon])
    final_lbl_val = final_lbl_train
    final_lbl_test = final_lbl_train
    final_p_train = final_lbl_train
    final_p_val= final_p_train
    final_p_test = final_p_train
elif dataset=="nasdaq":
    window_size = 180
    data = np.asarray(pd.read_csv(data_path+".csv"))
    knowledge_preds = np.asarray(pd.read_csv(knowledge_pred_path,header=None))
    n_val = 4056       # index from where validation set starts
    n_test = 2028      # index from where validation set starts
    data_length = data.shape[1]
    t_size=data.shape[0]
    output = np.zeros((n_test,data_length))
    final_in_train = np.zeros([1,window_size+horizon,1])
    final_in_val =final_in_train
    final_in_test=final_in_train
    final_lbl_train = np.zeros([1,horizon])
    final_lbl_val = final_lbl_train
    final_lbl_test = final_lbl_train
    final_p_train = final_lbl_train
    final_p_val= final_p_train
    final_p_test = final_p_train
    
else:
    window_size = 144    
    data = np.asarray(pd.read_csv(data_path+".txt",header=None))
    knowledge_preds = np.asarray(pd.read_csv(knowledge_pred_path,header=None))
    n_val = 3947       # index from where validation set starts
    n_test = 1973      # index from where validation set starts
    data_length = data.shape[1]
    t_size=data.shape[0]
    output = np.zeros((n_test,data_length))
    final_in_train = np.zeros([1,window_size+horizon,1])
    final_in_val =final_in_train
    final_in_test=final_in_train
    final_lbl_train = np.zeros([1,horizon])
    final_lbl_val = final_lbl_train
    final_lbl_test = final_lbl_train
    final_p_train = final_lbl_train
    final_p_val= final_p_train
    final_p_test = final_p_train

In [117]:
# n_val=2880
# n_test=1440
# data_length = data.shape[1]
# t_size=data.shape[0]
# output = np.zeros((n_test,data_length))
# final_in_train = np.zeros([1,window_size+horizon,1])
# final_in_train = np.zeros([1,window_size+horizon])

# final_in_val =final_in_train
# final_in_test=final_in_train
# final_lbl_train = np.zeros([1,horizon])
# final_lbl_val = final_lbl_train
# final_lbl_test = final_lbl_train
# final_p_train = final_lbl_train
# final_p_val= final_p_train
# final_p_test = final_p_train

#------------------for sota comparison-------------------
if dataset=='traffic':

    with tqdm(total=data_length) as pbar:
        for i in range(data_length):
            current_row= data[:,i]

            train = current_row[:-n_val]
            val = current_row[-(n_val+window_size):-n_test]
            test = current_row[-(n_test+window_size):]


    #         n_reduced=int(0.5*train.size)
    #         train=train[n_reduced:]
            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            if dataset !='traffic':

                current_pred= knowledge_preds[:(t_size-window_size),i]
            else:
                current_pred= knowledge_preds[:,i]
            train_p = current_pred[:-n_val] 
    #         train_p = current_pred[(n_reduced):-n_val]  
            val_p = current_pred[-n_val:-n_test]
            test_p = current_pred[-n_test:]
            train_pred = make_k_input(train_p,horizon)
            val_pred = make_k_input(val_p,horizon)
            test_pred = nonov_make_k_input(test_p,horizon)

            train_x=np.append(train_sequence[0],train_pred,axis=1)
            val_x=np.append(val_sequence[0],val_pred,axis=1)
            test_x=np.append(test_sequence[0],test_pred,axis=1)


            final_in_train =np.append(final_in_train,train_x,axis=0)
            final_lbl_train = np.append(final_lbl_train,train_sequence[1],axis=0)


            final_in_val =np.append(final_in_val,val_x,axis=0)
            final_lbl_val = np.append(final_lbl_val,val_sequence[1],axis=0)


            final_in_test =np.append(final_in_test,test_x,axis=0)
            final_lbl_test = np.append(final_lbl_test,test_sequence[1],axis=0)






            final_p_train =np.append(final_p_train,train_pred,axis=0)


            final_p_val =np.append(final_p_val,val_pred,axis=0)


            final_p_test =np.append(final_p_test,test_pred,axis=0)



            pbar.update(1)
# #----------------------------with normalization----------------------------------------------------        
        
else:
    with tqdm(total=data_length) as pbar:
    for i in range(data_length):
        current_row= data[:,i]

        train = current_row[:-n_val]
        n_reduced=int(0.5*train.size)
        train=train[n_reduced:]
        val = current_row[-(n_val+window_size):-n_test]
        test = current_row[-(n_test+window_size):]
        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        temp_train_x=train_sequence[0]    
        min_in_train = temp_train_x.min(1).reshape(temp_train_x.shape[0],1)
        max_in_train= temp_train_x.max(1).reshape(temp_train_x.shape[0],1)
        denom_train = (max_in_train-min_in_train)
        a = np.where(denom_train == 0)[0]
        denom_train[a] = max_in_train[a] 
        a = np.where(denom_train == 0)[0]
        if a.size >0:
            denom_train[a]=1
        temp_train_x = (temp_train_x-min_in_train)/denom_train
        temp_train_x=temp_train_x.reshape(temp_train_x.shape[0],temp_train_x.shape[1],1)
        temp_train_y=(train_sequence[1]-min_in_train)/denom_train


        temp_val_x=val_sequence[0]    
        min_in_val = temp_val_x.min(1).reshape(temp_val_x.shape[0],1)
        max_in_val= temp_val_x.max(1).reshape(temp_val_x.shape[0],1)
        denom_val = (max_in_val-min_in_val)
        a = np.where(denom_val == 0)[0]
        denom_val[a] = max_in_val[a] 
        a = np.where(denom_val == 0)[0]
        if a.size >0:
            denom_val[a]=1
        temp_val_x = (temp_val_x-min_in_val)/denom_val
        temp_val_x=temp_val_x.reshape(temp_val_x.shape[0],temp_val_x.shape[1],1)
        temp_val_y=(val_sequence[1]-min_in_val)/denom_val


        temp_test_x=test_sequence[0]    
        min_in_test = temp_test_x.min(1).reshape(temp_test_x.shape[0],1)
        max_in_test= temp_test_x.max(1).reshape(temp_test_x.shape[0],1)
        denom_test = (max_in_test-min_in_test)
        a = np.where(denom_test == 0)[0]
        denom_test[a] = max_in_test[a] 
        a = np.where(denom_test == 0)[0]
        if a.size >0:
            denom_test[a]=1
        temp_test_x = (temp_test_x-min_in_test)/denom_test
        temp_test_x=temp_test_x.reshape(temp_test_x.shape[0],temp_test_x.shape[1],1)
        temp_test_y=(test_sequence[1]-min_in_test)/denom_test



        current_pred= knowledge_preds[:(t_size-window_size),i]
        current_pred= knowledge_preds[n_reduced:(t_size-window_size),i]
        train_p = current_pred[:-n_val]                                        
        val_p = current_pred[-n_val:-n_test]
        test_p = current_pred[-n_test:]
        train_pred = make_k_input(train_p,horizon)
        val_pred = make_k_input(val_p,horizon)
        test_pred = nonov_make_k_input(test_p,horizon)

        temp_train_p_x=train_pred
        min_in = temp_train_p_x.min(1).reshape(temp_train_p_x.shape[0],1)
        max_in = temp_train_p_x.max(1).reshape(temp_train_p_x.shape[0],1)
        denom = (max_in-min_in)
        a = np.where(denom == 0)[0]
        denom[a] = max_in[a] #--------------------------------------check
        a = np.where(denom == 0)[0]
        if len(a)>0:
            denom[a]=1
        temp_train_p_x = (temp_train_p_x-min_in_train)/denom_train 
#         temp_train_p_x[a] =0.5

        final_p_train =np.append(final_p_train,temp_train_p_x,axis=0)

        temp_val_p_x=val_pred
        min_in = temp_val_p_x.min(1).reshape(temp_val_p_x.shape[0],1)
        max_in = temp_val_p_x.max(1).reshape(temp_val_p_x.shape[0],1)
        denom = (max_in-min_in)
        a = np.where(denom == 0)[0]
        denom[a] = max_in[a] #--------------------------------------check
        a = np.where(denom == 0)[0]
        if len(a)>0:
            denom[a]=1
        temp_val_p_x = (temp_val_p_x-min_in_val)/denom_val
        temp_val_p_x[a] = 0.5

        final_p_val =np.append(final_p_val,temp_val_p_x,axis=0)

        temp_test_p_x=test_pred
        min_in = temp_test_p_x.min(1).reshape(temp_test_p_x.shape[0],1)
        max_in = temp_test_p_x.max(1).reshape(temp_test_p_x.shape[0],1)
        denom = (max_in-min_in)
        a = np.where(denom == 0)[0]
        denom[a] = max_in[a] #--------------------------------------check
        a = np.where(denom == 0)[0]
        if len(a)>0:
            denom[a]=1
        temp_test_p_x = (temp_test_p_x-min_in_test)/denom_test
        temp_test_p_x[a] = 0.5



        final_p_test =np.append(final_p_test,temp_test_p_x,axis=0)
        
        
        
        temp_train_p_x =temp_train_p_x.reshape(temp_train_p_x.shape[0],temp_train_p_x.shape[1],1)
        temp_train_x=np.append(temp_train_x,temp_train_p_x,axis=1)
        final_in_train =np.append(final_in_train,temp_train_x,axis=0)
        final_lbl_train = np.append(final_lbl_train,temp_train_y,axis=0)
        
        temp_val_p_x =temp_val_p_x.reshape(temp_val_p_x.shape[0],temp_val_p_x.shape[1],1)
        temp_val_x=np.append(temp_val_x,temp_val_p_x,axis=1)
        final_in_val =np.append(final_in_val,temp_val_x,axis=0)
        final_lbl_val = np.append(final_lbl_val,temp_val_y,axis=0)
        
        temp_test_p_x =temp_test_p_x.reshape(temp_test_p_x.shape[0],temp_test_p_x.shape[1],1)
        temp_test_x=np.append(temp_test_x,temp_test_p_x,axis=1)
        final_in_test =np.append(final_in_test,temp_test_x,axis=0)
        final_lbl_test = np.append(final_lbl_test,temp_test_y,axis=0)

        pbar.update(1)
final_in_train = final_in_train[1:,:,:]
final_in_val = final_in_val[1:,:,:]
final_in_test = final_in_test[1:,:,:]
final_lbl_train = final_lbl_train[1:,:]
final_lbl_val = final_lbl_val[1:,:]
final_lbl_test = final_lbl_test[1:,:]
final_p_train = final_p_train[1:,:]
final_p_val = final_p_val[1:,:]
final_p_test = final_p_test[1:,:]

)


100%|██████████| 82/82 [00:23<00:00,  2.19it/s]


In [8]:
final_p_train.shape

(406458, 12)

In [9]:
final_in_train.shape

(406458, 156, 1)

In [118]:
tf.reset_default_graph()
K.clear_session()  





input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')
input_pred=Input(batch_shape=(None,horizon),name='input_pred')

if dataset!='traffic':
    branch_0 = Conv1D(4,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform())(input_data)
    branch_1 = Conv1D(32,3, strides=1,dilation_rate=1,padding='same',activation='relu',kernel_initializer=glorot_uniform())(branch_0)
    branch_2 = Conv1D(64,3, strides=1,dilation_rate=1, padding='same',activation='relu',kernel_initializer=glorot_uniform())(branch_1)

    branch_4=Flatten()(branch_2)
    net= Dense(128,activation='relu')(branch_4)
    net= Dense(horizon,activation='relu')(net)
    net=add([net,input_pred])
#         ,activity_regularizer=regularizers.l2(0.01)
else:
    branch_0 = LSTM(512,activation='sigmoid',kernel_initializer=glorot_uniform(1),return_sequences=True,name='lstm_1')(input_data)
    branch_1 = LSTM(64,activation='relu',kernel_initializer=glorot_uniform(2),return_sequences=True,name='lstm_2')(branch_0)
    branch_2 = LSTM(16,activation='relu',kernel_initializer=glorot_uniform(3),name='lstm_3')(branch_1)
    net= Dense(horizon,name='dense_final2')(branch_2)
    net=add([net,input_pred])

model=Model(inputs=[input_data,input_pred],outputs=net)


In [119]:
callback = ModelCheckpoint(filepath='/home/chatta/logic_rules/model_checkpoints/'+dataset+'/h'+str(horizon)+'/dnn_50_1.h5',monitor='val_loss',save_best_only=True)

model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))
hist=model.fit({'input_data':final_in_train,'input_pred':final_p_train},final_lbl_train,validation_data=[[final_in_val,final_p_val],final_lbl_val],callbacks=[callback],batch_size=4*512,shuffle=False, epochs=50,verbose=1)

Train on 1481494 samples, validate on 165886 samples
Epoch 1/50
1481494/1481494 [==============================] - 14s 10us/step - loss: 0.0305 - val_loss: 0.0233
Epoch 2/50
1481494/1481494 [==============================] - 14s 9us/step - loss: 0.0305 - val_loss: 0.0233
Epoch 3/50
1481494/1481494 [==============================] - 14s 9us/step - loss: 0.0305 - val_loss: 0.0233
Epoch 4/50
1481494/1481494 [==============================] - 14s 9us/step - loss: 0.0305 - val_loss: 0.0233
Epoch 5/50
1481494/1481494 [==============================] - 14s 9us/step - loss: 0.0305 - val_loss: 0.0233
Epoch 6/50
1481494/1481494 [==============================] - 14s 9us/step - loss: 0.0305 - val_loss: 0.0233
Epoch 7/50
1481494/1481494 [==============================] - 14s 9us/step - loss: 0.0305 - val_loss: 0.0233
Epoch 8/50
1481494/1481494 [==============================] - 14s 9us/step - loss: 0.0305 - val_loss: 0.0233
Epoch 9/50
1481494/1481494 [==============================] - 14s 9us/step

In [123]:
model=load_model('/home/chatta/logic_rules/model_checkpoints/'+dataset+'/h'+str(horizon)+'/dnn_50_best.h5')

In [124]:
for i in range(data_length):
    
    if dataset=="traffic":
        current_row= data[:,i]

        test = current_row[-(n_test+window_size):]
        test_sequence = nonov_make_input(test,window_size,horizon)

        

        current_pred= knowledge_preds[:,i]
        test_p = current_pred[-n_test:]
        test_pred = nonov_make_k_input(test_p,horizon)
        norm_p_test_x=p_window_normalize(test_pred) 

        test_x=np.append(test_sequence[0],test_pred,axis=1)






        pred = model.predict({'input_data':test_x.reshape(test_x.shape[0],test_x.shape[1],1),'input_pred':test_pred})
    #     pred=pred*denom+min_in
        prediction = pred.flatten()[:n_test]

        output[:,i]=np.transpose(prediction)
    else:
        current_row= data[:,i]
        series_d = current_row
        test = series_d[-(n_test+window_size):]
        test_sequence = nonov_make_input(test,window_size,horizon)

        temp_test_x=test_sequence[0]    

        min_in_test = temp_test_x.min(1).reshape(temp_test_x.shape[0],1)
        max_in_test = temp_test_x.max(1).reshape(temp_test_x.shape[0],1)
        denom_test = (max_in_test-min_in_test)
        a = np.where(denom_test == 0)[0]
        denom_test[a] = max_in_test[a]  #----------------------------------check
        temp_test_x = (temp_test_x-min_in_test)/denom_test
        temp_test_x=temp_test_x.reshape(temp_test_x.shape[0],temp_test_x.shape[1],1)
        temp_test_y=(test_sequence[1]-min_in_test)/denom_test

        current_pred= knowledge_preds[:,i]
        
        test_p = current_pred[-n_test:]
        test_k_pred = nonov_make_k_input(test_p,horizon)

        temp_test_p_x=test_k_pred
        min_in = temp_test_p_x.min(1).reshape(temp_test_p_x.shape[0],1)
        max_in = temp_test_p_x.max(1).reshape(temp_test_p_x.shape[0],1)
        denom = (max_in-min_in)
        a = np.where(denom == 0)[0]
        denom[a] = max_in[a] #--------------------------------------check
        a = np.where(denom == 0)[0]
        if len(a)>0:
            denom[a]=1
        temp_test_p_x = (temp_test_p_x-min_in_test)/denom_test
        temp_test_p_x[a] = 0.5

        temp_test_p_x1 =temp_test_p_x.reshape(temp_test_p_x.shape[0],temp_test_p_x.shape[1],1)
        temp_test_x=np.append(temp_test_x,temp_test_p_x1,axis=1)
#         final_in_test =np.append(final_in_test,temp_test_x,axis=0)
#         final_lbl_test = np.append(final_lbl_test,temp_test_y,axis=0)

        pred = model.predict({'input_data':temp_test_x, 'input_pred':temp_test_p_x})
        prediction = pred*(max_in_test-min_in_test)+min_in_test
        prediction = prediction.flatten()[:n_test]

        output[:,i]=np.transpose(prediction)
    
# if dataset=="concept":
#     # preds=model.predict({'input_data':final_in_test,'input_pred':final_p_test})*denom_test+min_test

In [125]:
temp1, temp2= metrics(output[-n_test:,:],data[-n_test:,:])
[temp1,temp2]

[0.24697988199680967, 0.06291757391341506]

In [145]:
# np.savetxt('/home/chatta/logic_rules/results/pacf_preds/traffic_30_std/h1/preds_noisy.csv',knowledge_preds, fmt='%1.5f',delimiter=',')